# Excercise 5

## Myokyung Han (20224230)

In [1]:
import pandas as pd
from pandas.tseries.offsets import YearEnd

$$
    \text{Real XR} = \frac{S^{PPP}}{S}
$$

In [2]:
# update frequncy is low
# 2021의 데이터가 없다....

# 직접 real exchange rate을 구한다.


$$
    S^{PPP} = \frac{{P(CAD)}}{P(USD)}
$$

In [3]:
## PPP는 유동성이 적으므로 다운받고 market exchange는 daily로 받는다. 

In [4]:
pd.read_excel('./data/world_ppp_xr.xls')
# .head()

FileNotFoundError: [Errno 2] No such file or directory: './data/world_ppp_xr.xls'

In [ ]:
def load_deepsearch_data(filepath, column):
    df = pd.read_excel(filepath)
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.set_index('날짜')
    df.columns = [column]
    
    return df
    

In [ ]:
df_xr = load_deepsearch_data('./data/krw_usd_xr.xlsx', 'KRW/USD')

In [ ]:
df_monthly = df_xr.resample('M').last()

In [ ]:
df_monthly = df_monthly.loc['1998-12-31':]

In [ ]:
df_monthly.head()

In [ ]:
def load_world_bank_data(filepath, column):
    df = pd.read_excel(filepath, sheet_name = 'Data', skiprows = 3)
    
    df = df.set_index('Country Code')
    df = df.loc['KOR'].to_frame()
    
    df = df.iloc[3:]
    df.dropna(inplace=True)
    df['KOR'] = df['KOR'].astype(float)
    df['date'] = pd.to_datetime(df.index)+YearEnd(0)
    df.set_index('date', inplace=True)
    df.columns = [column]
    return df
      
    

In [ ]:
df_real_xr = load_world_bank_data('./data/world_real_xr.xls', 'Real XR')

In [ ]:
df_ppp_xr = load_world_bank_data('./data/world_ppp_xr.xls', 'PPP XR')

In [ ]:
df_ppp_xr.head(3)

In [ ]:
df_ppp_xr.tail(3)

## Merge dataframes

In [ ]:
df_merged = df_monthly.join(df_real_xr)

In [ ]:
df_merged= df_merged.join(df_ppp_xr)

In [ ]:
df_real_xr

In [ ]:
df_ppp_xr

In [ ]:
df_merged.head()

In [ ]:
df_merged.tail()

In [ ]:
df_real_xr.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = (10,4)

In [ ]:
plt.plot(df_merged['KRW/USD'], label = 'Market XR')
plt.plot(df_merged['PPP XR'].dropna(), label = 'PPP XR')
plt.legend()
plt.show()

In [ ]:
df_merged['PPP XR'] = df_merged['PPP XR'].fillna(method='ffill')

In [ ]:
df_merged['New Real XR'] = \
    df_merged['PPP XR']/df_merged['KRW/USD']

In [ ]:
df_merged.head()

In [ ]:
df_merged['Real XR'] = df_merged['Real XR'].fillna(method='ffill')

In [ ]:
plt.plot(df_merged['New Real XR'], label = 'New Real XR')
plt.plot(df_merged['Real XR'].shift(-12), label = 'Original World Bank')
plt.legend()
plt.show()

### Compute annual returns in the currency market

In [ ]:
df_merged['ann_ret'] = \
    df_merged['KRW/USD'].pct_change(periods=12)*100

In [ ]:
df_merged['F12.ann_ret'] = \
    df_merged['ann_ret'].shift(-12)

In [ ]:
df_merged.head(15)

### Use OLS!!!

In [ ]:
import statsmodels.api as sm

In [ ]:
df = df_merged[['New Real XR', 'F12.ann_ret']]
df = df.dropna()

In [ ]:
X = df['New Real XR']
X = sm.add_constant(X)

In [ ]:
Y = df['F12.ann_ret']

In [ ]:
model = sm.OLS(Y,X)

In [ ]:
results = model.fit()

In [ ]:
results.summary()

## Forecast XR

$$
y = \alpha + \beta x
$$

$$
\text{ret}_{t, t+12} = \alpha + \beta \, \text{Real XR}_t +\epsilon
$$

In [ ]:
results.params

In [ ]:
dir(results)

In [ ]:
results.params

In [ ]:
last_obs = df_merged.iloc[-1]

In [ ]:
last_xr = last_obs['KRW/USD']
last_year_xr = last_obs['New Real XR']

In [ ]:
last_xr

In [ ]:
last_year_xr

### method 1: simple , straightfoward, stupid

In [ ]:
alpha = results.params['const']
beta = results.params['New Real XR']

In [ ]:
alpha+beta*last_year_xr

### method 2: numpy 

In [ ]:
import numpy as np

In [ ]:
exog = [1, last_year_xr]

In [ ]:
np.dot(results.params, exog)

### method 3: predict

In [ ]:
results.predict(exog = exog)

In [ ]:
help(results.predict)

### confidence interval of our predictions

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
wls_prediction_std(results, exog = exog, alpha=0.05)

# alpha 0.05 >> 95% ...,,,,,,,,...

# predicted stdev, interval - lower, interval - upper 

In [ ]:
type(exog)

In [ ]:
for alpha in [.05, .25, .50]:
    prstd, iv_l, iv_u = wls_prediction_std(results, exog = exog, alpha=alpha)
    
    print('Confidence interval at alpha {:.2f} : {:.2f} % ~ {:.2f}%'.format( alpha, iv_l[0], iv_u[0]))

In [ ]:
pred_ret = results.predict(exog = exog )

In [ ]:
last_xr * (1+pred_ret[0]/100)

In [ ]:
last_xr

In [ ]:
for alpha in [.05, .25, .50]:
    prstd, iv_l, iv_u = wls_prediction_std(results, exog = exog, alpha=alpha)
    
    print('Confidence interval at alpha {:.2f} : {:.0f}  ~ {:.0f}'.format( alpha, last_xr*(1+iv_l[0]/100), last_xr*(1+iv_u[0]/100)))